# garbage_classifier

This notebook classifies website text snippets into useful or not (i.e., garbage) using transfer learning starting from an existing hugging face model
* Get a model checkpoint for an encoder model 
* Use reinforcement learning to apply the model on a new classification problem (EAGER website data) with limited new trained data
* Apply new head of model to full EAGER corpus to come up with mixes of models
* Metrics and model registered through a combination of comet.ml and tensorboard 

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jul  7 18:13:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install and import libraries

In [2]:
COMET_PROJECT_NAME = "eager-garbage-classifier"

In [3]:
# check environment
import sys
IN_COLAB = 'google.colab' in sys.modules
print (IN_COLAB)

True


In [4]:
# colab file system setup 
if IN_COLAB: 
    !git clone https://github.com/euphonic/EAGER.git
    !pwd
    !mkdir /content/logs

Cloning into 'EAGER'...
remote: Enumerating objects: 19713, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 19713 (delta 247), reused 364 (delta 220), pack-reused 19308
Receiving objects: 100% (19713/19713), 370.92 MiB | 29.80 MiB/s, done.
Resolving deltas: 100% (5965/5965), done.
Checking out files: 100% (5176/5176), done.
/content


In [5]:
# mount google drive if in colab
drive_path = '/content/drive/'

if IN_COLAB:  
    from google.colab import drive
    drive.mount(drive_path, force_remount=True)

Mounted at /content/drive/


In [6]:
# install huggingface and other modules if in colab
if IN_COLAB: 
    !pip install transformers
    !pip install datasets
    !pip install python-dotenv
    !pip install comet_ml==3.30.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 57.4 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 212 kB 57.2 MB/s 
     |████████████████████████████████| 140 kB 61.4 MB/s 
     |████████████████████████████████| 1.1 MB 55.0 MB/s 
     |████████████████████████████████| 127 kB 55.4 MB/s 
     |████████████████████████████████| 144 kB 63.3 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |█████████████

In [8]:
from comet_ml import Experiment
from dotenv import load_dotenv

# setup comet_ml experiment
if IN_COLAB: 
    # read env file from Google drive 
    env_file = drive_path + 'MyDrive/raaste-config/.env'
    comet_config_file = drive_path + 'MyDrive/raaste-config/.comet.config'
    load_dotenv(env_file)

In [24]:
# ml libraries
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding, AutoConfig
from datasets import Dataset
import datasets
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
import pandas as pd

# other
import numpy as np
import gzip
import tarfile
import datetime

In [10]:
# load tensorboard 
%load_ext tensorboard

## Garbage classifier
keep test == 1, discard == 0

In [12]:
# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [13]:
firm_file_location = '/content/EAGER/data/modeling/garbage/garbage_classifier_input.csv'
input_df = pd.read_csv(firm_file_location)

In [14]:
non_null_df = input_df[~ input_df['sample_text'].isnull() ]
non_null_df.shape

(5601, 2)

In [15]:
dataset = Dataset.from_pandas(non_null_df, split='train')
dataset.cast_column("of_interest", datasets.Value('int8'))

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['sample_text', 'of_interest', '__index_level_0__'],
    num_rows: 5601
})

In [16]:
# 70% train, 30% test + validation
train_test_dataset = dataset.train_test_split(test_size=0.3)
# Split the 30% test + valid in half test, half valid
test_valid_dataset = train_test_dataset['test'].train_test_split(test_size=0.30)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = datasets.DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid_dataset['test'],
    'valid': test_valid_dataset['train']})

In [17]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['sample_text', 'of_interest', '__index_level_0__'],
        num_rows: 3920
    })
    test: Dataset({
        features: ['sample_text', 'of_interest', '__index_level_0__'],
        num_rows: 505
    })
    valid: Dataset({
        features: ['sample_text', 'of_interest', '__index_level_0__'],
        num_rows: 1176
    })
})

In [18]:
def tokenize_function(x):
  return tokenizer(x["sample_text"], truncation=True, max_length=100)

In [19]:
tokenized_dataset = train_test_valid_dataset.map(tokenize_function, batched=True, batch_size=None)

Parameter 'function'=<function tokenize_function at 0x7faee24c1050> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['sample_text', 'of_interest', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3920
    })
    test: Dataset({
        features: ['sample_text', 'of_interest', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 505
    })
    valid: Dataset({
        features: ['sample_text', 'of_interest', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1176
    })
})

In [33]:
samples = tokenized_dataset["train"].to_dict()
samples = {k: v for k, v in samples.items() if k not in ["__index_level_0__", "sample_text"]}
for k, v in samples.items(): 
  print (k, v[0:5])

of_interest [0, 0, 1, 0, 1]
input_ids [[101, 26624, 1708, 102], [101, 2892, 102], [101, 1110, 170, 2616, 118, 3903, 117, 10682, 5863, 1105, 3317, 1718, 3547, 1105, 1555, 11482, 119, 1284, 1202, 1136, 4845, 1114, 1412, 5793, 119, 1284, 23200, 4099, 1105, 1579, 3244, 8132, 25557, 1785, 1105, 8066, 2400, 119, 9612, 1487, 117, 1195, 3689, 8156, 7995, 1111, 1412, 5793, 117, 12619, 2299, 3317, 23510, 119, 1284, 5622, 1106, 1532, 2012, 15605, 1114, 1412, 5793, 117, 1781, 2982, 1976, 1105, 19723, 1194, 1718, 1105, 1154, 2616, 3903, 5863, 119, 102], [101, 149, 11680, 12649, 1708, 117, 111, 139, 11607, 9244, 2591, 10783, 8900, 102], [101, 157, 1182, 1592, 1233, 6552, 1111, 2163, 4683, 102]]
token_type_ids [[0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [34]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=100, return_tensors="tf")

In [35]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

{'attention_mask': TensorShape([3920, 100]),
 'input_ids': TensorShape([3920, 100]),
 'of_interest': TensorShape([3920]),
 'token_type_ids': TensorShape([3920, 100])}

In [36]:
# config
config = AutoConfig.from_pretrained(checkpoint)
config.num_labels=1
config.hidden_dropout_prob = 0.2
config.attention_probs_dropout_prob = 0.2
print (type(config))

# model
model = TFAutoModelForSequenceClassification.from_config(config)
print (type(model))

<class 'transformers.models.bert.configuration_bert.BertConfig'>
<class 'transformers.models.bert.modeling_tf_bert.TFBertForSequenceClassification'>


In [ ]:
num_epochs = 20
batch_sizes = [64]

for bs in batch_sizes: 
  # read config file from git repo 
  experiment = Experiment(project_name=COMET_PROJECT_NAME)
  with experiment.train():
    experiment.log_parameter("batch_size", bs)

  print ('batch_size', bs)

  tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols="of_interest",
    shuffle=True,
    collate_fn=data_collator,
    batch_size=bs,
  )

  tf_validation_dataset = tokenized_dataset["valid"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols="of_interest",
    shuffle=False,
    collate_fn=data_collator,
    batch_size=bs,
  )

  # The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
  # by the total number of epochs
  num_train_steps = len(tf_train_dataset) * num_epochs
  lr_scheduler = PolynomialDecay(
      initial_learning_rate=5e-5, end_learning_rate=5e-10, decay_steps=num_train_steps
  )

  opt = Adam(learning_rate=lr_scheduler)

  log_dir = "/content/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)    

  early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

  model.fit(tf_train_dataset, validation_data=tf_validation_dataset, 
        epochs=num_epochs, callbacks=[tensorboard_callback, early_stopping_callback])
  
  experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET ERROR: Run will not be logged 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/comet_ml/experiment.py", line 893, in _start
    self.alive = self._setup_streamer()
  File "/usr/local/lib/python3.7/dist-packages/comet_ml/_online.py", line 282, in _setup_streamer
    results = self._authenticate()
  File "/usr/local/lib/python3.7/dist-packages/comet_ml/_online.py", line 332, in _authenticate
    self.project_name, self.workspace
  File "/usr/local/lib/python3.7/dist-packages/comet_ml/connection.py", line 582, in get_run_id
    "libVersion": _comet_version(),
  File "/usr/local/lib/python3.7/dist-packages/comet_ml/connection.py", line 123, in _comet_version
    return comet_ml.__version__
Att

batch_size 64
Epoch 1/20
62/62 [==============================] - 86s 1s/step - loss: 0.3920 - accuracy: 0.8286 - val_loss: 0.3411 - val_accuracy: 0.8359
Epoch 2/20
62/62 [==============================] - 65s 1s/step - loss: 0.3165 - accuracy: 0.8663 - val_loss: 0.2972 - val_accuracy: 0.8690
Epoch 3/20
62/62 [==============================] - 65s 1s/step - loss: 0.2333 - accuracy: 0.9064 - val_loss: 0.2996 - val_accuracy: 0.8784
Epoch 4/20
52/62 [========================>.....] - ETA: 7s - loss: 0.1551 - accuracy: 0.9369

## Register model

In [ ]:
  experiment.end()git